In [119]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from torch.autograd import Variable
from torch import Tensor, optim, nn
import wandb
from tqdm import tqdm
wandb.login()

True

In [120]:
config = {
    "learning_rate": 0.02,
    "architecture": "NN",
    "dataset": "static_1.0",
    "epochs": 10,
    "num_classes": 2,
    "batch_size": 64,
    "num_layers": 2,
    "hidden_size": 64,
    "dropout_prob": 0.2,
    "input_size": 229,
    "output_size": 2,
    "optimizer": "Adam",
    "loss": "CrossEntropyLoss",
    "activation": "ReLU",
    "initializer": "Xavier",
    "regularization": "L2",
    "regularization_lambda": 0.01,
    }

In [121]:
def model_pipeline(hyperparameters):
    with wandb.init(project="leaguify", config=hyperparameters):
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config

      # make the model, data, and optimization problem
      model, train_loader, test_loader, criterion, optimizer = make(config)
      print(model)

      # and use them to train the model
      train(model, train_loader, criterion, optimizer, config)

      # and test its final performance
      test(model, test_loader)

    return model

In [137]:
class StaticDataset(Dataset):
    def __init__(self, data_dir, transform=None, target_transform=None):
        self.data = torch.tensor(np.load(data_dir)[:, :-1], dtype=torch.float32, device=device)
        self.labels = torch.tensor(np.load(data_dir)[:, -1], dtype=torch.int64, device=device)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx, 1:]
        label = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        if self.target_transform:
            label = self.target_transform(label)
        return sample, label

In [138]:
def make(config):
    train, test = get_data(train=True), get_data(train=False)
    train_loader = make_loader(train, batch_size=config.batch_size)
    test_loader = make_loader(test, batch_size=config.batch_size)

    model = NeuralNetwork(config.input_size, config.hidden_size, config.num_layers, config.dropout_prob, config.num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    return model, train_loader, test_loader, criterion, optimizer

In [139]:
def get_data(slice=5, train=True):
    if train:
        full_dataset = StaticDataset('../data/train_static.npy')
    else:
        full_dataset = StaticDataset('../data/test_static.npy')
    sub_dataset = torch.utils.data.Subset(full_dataset, range(0, len(full_dataset), slice))
    return sub_dataset

In [140]:
def make_loader(dataset, batch_size=64):
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=0)

In [141]:
device = (
    "cuda" if torch.cuda.is_available()
    else "cpu"
)
if torch.cuda.is_available():
    print(f'PyTorch version: {torch.__version__}')
    print('*' * 10)
    print(f'_CUDA version: ')
    !nvcc --version
    print('*' * 10)
    print(f'CUDNN version: {torch.backends.cudnn.version()}')
    print(f'Available GPU devices: {torch.cuda.device_count()}')
    print(f'Device Name: {torch.cuda.get_device_name()}')
print(f"Using {device} device")

Using cpu device


In [142]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob, classes=2):
        super(NeuralNetwork, self).__init__()
        self.dropout = nn.Dropout(dropout_prob)
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 2),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [143]:
def train(model, loader, criterion, optimizer, config):
    wandb.watch(model, criterion, log="all", log_freq=10)

    total_batches = len(loader) * config.epochs
    example_count = 0
    batch_count = 0
    for epoch in tqdm(range(config.epochs)):
        for _, (matches, labels) in enumerate(loader):
            loss = train_batch(matches, labels, model, optimizer, criterion)
            example_count += len(matches)
            batch_count += 1
            if (batch_count + 1) % 25 == 0:
                train_log(loss, example_count, epoch)


In [144]:
def train_batch(matches, labels, model, optimizer, criterion):
    optimizer.zero_grad()
    output = model(matches)
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()
    return loss

In [145]:
def train_log(loss, example_count, epoch):
    wandb.log({"epoch": epoch, "loss": loss}, step=example_count)
    print(f"Loss after {str(example_count).zfill(5)} examples: {loss:.3f}")

In [146]:
def test(model, test_loader):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        for matches, labels in test_loader:
            matches, labels = matches.to(device), labels.to(device)
            outputs = model(matches)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Accuracy of the model on the {total} " +
              f"test matches: {correct / total:%}")

        wandb.log({"test_accuracy": correct / total})

In [147]:
model = model_pipeline(config)

NeuralNetwork(
  (dropout): Dropout(p=0.2, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=229, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=2, bias=True)
    (5): ReLU()
  )
)


100%|██████████| 10/10 [00:00<00:00, 75.97it/s]

torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
torch.float32 torch.int64
Accuracy of the model on the 14 test images: 57.142857%


test_accuracy,▁
test_accuracy,0.57143
